In [1]:
#Structure of the notebook:
    #many functions are defined
        #I put the function descriptions in their definitions
    #all the functions are called from a single cell, (last cell)
    #entire notebook can be run remotely with: 
        #%run C:\\Users\\walke\\Downloads\\Sample_loading_gaze_data_DBSCAN_Agglomerative-Looping.ipynb
        #NOTE: jupyter notebooks currently does not show plots via this method
    #Everything, I believe, is as it should be EXCEPT for one cell where 
        #the handling of '0' in the dataset is returning Nulls
        #I have instituted a temporary solution particular to each dataset
        #But this is the key problem I will tackle so that we can run the code
        #on all the subject with ease
        
#imports should all be here
import pickle
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import time
from scipy import stats
import os
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler

In [2]:
#gaze_stamp is just combining the gaze points and timestamp into a single array
def initialize_gaze_stamp():
    gaze_stamp = np.ndarray(shape=(3,len(gaze.transpose())), dtype=np.float64, order='C')
    #create single array with both xy gaze and timestamp
    gaze_stamp[0:2] = gaze
    gaze_stamp[2] = timestamp
    return gaze_stamp

In [3]:
#basic visualization of gaze_stamp
def initial_vis():
    # Example visualizations, histograms for gaze points. 
    f, axarr = plt.subplots(1, 2,sharex=True, figsize=(20,5))

    spidx=0 # set subplot index.

    x_hist = axarr[spidx].hist(gaze[0,:],100) 
    axarr[spidx].set_title('Histogram of x-coordinates')
    axarr[spidx].set_xlabel('X-coordinates')
    axarr[spidx].set_ylabel('frequency')

    spidx=1 # set subplot index.

    x_hist = axarr[spidx].hist(gaze[1,:],100) 
    axarr[spidx].set_title('Histogram of y-coordinates')
    axarr[spidx].set_xlabel('Y-coordinates')
    axarr[spidx].set_ylabel('frequency')

    # Display gaze map.

    gaze_tp = gaze.transpose()
    X = gaze_stamp.transpose()
    plt.figure(figsize=(15,10))  # just to specify the figure size 
    plt.scatter(X[:,0],-X[:,1])  # reverse y coordinates since screen position start from upper left corner.
    

In [4]:
#dbscan on gaze_stamp
def my_dbscan():
    db = DBSCAN(eps=0.3, min_samples=10).fit(gaze.transpose())
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)

    # #############################################################################
    # Plot result
    import matplotlib.pyplot as plt

    # Black removed and is used for noise instead.
    unique_labels = set(labels)
    colors = [plt.cm.Spectral(each)
              for each in np.linspace(0, 1, len(unique_labels))]
    for k, col in zip(unique_labels, colors):
        if k == -1:
            # Black used for noise.
            col = [0, 0, 0, 1]
        class_member_mask = (labels == k)

        xy = gaze_stamp.transpose()[class_member_mask & core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                 markeredgecolor='k', markersize=25)

        xy = gaze_stamp.transpose()[class_member_mask & ~core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                 markeredgecolor='k', markersize=2)

    plt.title('Estimated number of clusters: %d' % n_clusters_)
    plt.show()
    return labels

In [5]:
#add dbscan label to give us gsln, (n denotes removing noise points. l stands for label)
def create_gsln(labels):
    #gsl == gaze, timeStamp, and label
    gsl = np.ndarray(shape=(4,len(gaze.transpose())), dtype=np.float64, order='C')

    gsl[3] = labels
    gsl[2] = timestamp
    gsl[0:2] = gaze
    gsln = gsl[:,gsl[3,:]!=-1]
    return gsln

In [6]:
#agglomerative clustering on glsn
def agglom_clustering(gsln):

    X = gsln.transpose()

    connectivity=None
    n_clusters=50
    #we probably only need to use one of these methods?
    for index, linkage in enumerate(('average',
                                         'complete',
                                         'ward',)):
        plt.subplot(1, 4, index + 1)
        model = AgglomerativeClustering(linkage=linkage,
                                        connectivity=None,
                                        n_clusters=n_clusters)
        t0 = time.time()
        model.fit(X)
        elapsed_time = time.time() - t0
        plt.scatter(X[:, 0], X[:, 1], c=model.labels_,
                    cmap=plt.cm.nipy_spectral)
        plt.axis('equal')
        plt.axis('on')

        plt.subplots_adjust(bottom=0, top=.89, wspace=0,
                            left=0, right=4)
    plt.show()
    return model.labels_

In [7]:
#plots 1 agglomerative result (gaze and label)
def plot_one_agglom(gsla):
    plt.subplot(1, 4, index + 1)
    plt.scatter(gsla[0, :], gsla[1, :], c=gsla[3],
                cmap=plt.cm.nipy_spectral)
    plt.title('linkage=%s\n(time %.2fs)' % (linkage, elapsed_time),
              fontdict=dict(verticalalignment='top'))
    plt.axis('equal')
    plt.axis('off')
    plt.subplots_adjust(bottom=-1, top=.89, wspace=0,
                        left=-5, right=1)
    plt.suptitle('n_cluster=%i, connectivity=%r' %
                 (n_clusters, connectivity is not None), size=25)
    plt.show()

In [8]:
#should be able to save various data types with various names
def save_results(gsla, fn):
    f_name = fn
    gaze_agg_list = []
    try:
        with open("D:\\Documents\\My_Code\\Christoforos\\saved_results\\" + f_name + ".txt", "rb") as fp:   # Unpickling
            gaze_agg_list = pickle.load(fp)
            print('len of gaze_agg_list: ', len(gaze_agg_list))

        #already have data at this point
        with open("D:\\Documents\\My_Code\\Christoforos\\saved_results\\" + f_name + ".txt", "wb") as fp:   #Pickling
            gaze_agg_list.append(gsla)
            pickle.dump(gaze_agg_list, fp)
        print("succesfully loaded previous data and added new data to it.")
        
    except Exception: 
        print('nothing to open; saving first file: ')

        with open("D:\\Documents\\My_Code\\Christoforos\\saved_results\\" + f_name  + ".txt", "wb") as fp:   #Pickling
            print("gaze_agg_list should be empty: ", gaze_agg_list)
            gaze_agg_list.append(gsla)
            pickle.dump(gaze_agg_list, fp)

In [9]:
#just to be sure we are not using old data 
def reset_save(file_name):
    fn = file_name
    dumby_list = []
    with open("D:\\Documents\\My_Code\\Christoforos\\saved_results\\" + fn + ".txt", "wb") as fp:   #Pickling
        pickle.dump(dumby_list, fp)
    print('WARNING!!! succesfully replaced file with dumby data')

In [10]:
#I am converting to np.array here as output. 
def load_results(file_name):
    fn = file_name
    var = []
    with open("D:\\Documents\\My_Code\\Christoforos\\saved_results\\" + fn + ".txt", "rb") as fp:   # Unpickling
        var = pickle.load(fp)
    
    var_arr = np.asarray(var)
    var_arr  = var_arr[0]
    
    return var_arr

In [11]:
#gaze with dbscan labels
def plt_dbscan_with_time(gsln):
    #dbscan labels with time
    sorted_dbscan_labels = sorted(gsln[3])
    plt.scatter(gsln[2], sorted_dbscan_labels)

In [12]:
#gaze with aglomerative labels
def plt_agglom_with_time(gsla):
    #agglomerative labels with time
    sorted_agglom_labels = sorted(gsla[3])
    plt.scatter(gsla[2],sorted_agglom_labels)

In [13]:
def find_last_element(i, gsl_arr):
    c = i
    if i == 41:
        print('erf')
    for c in range(i, len(gsl_arr[3]) - i):
        if gsl_arr[3,i] == gsl_arr[3,c]:
            max_index = c
    try:
        return max_index
    except:
        return i

In [14]:
def find_first_element(i, gsl_arr):
    c = i
    if i == 41:
        print('hjg')
    for c in range( len(gsl_arr[3]) ):
        if gsl_arr[3,i] == gsl_arr[3,c]:
            min_index = cX
            return min_index

In [15]:
#removes any overlap in labels, (Ensures they are grouped with no mixing)
#THE ASSIGNMENT ERROR IS HERE
    #gsl_d[3,temp_min:temp_max] = gsl_d[3,i] appears to be the issue
        #how to resolve...?
def remove_overlap(gsl_arr):
    gsl_d = np.copy(gsla)#temporary array

    for i in range(len(gsl_d[3])):
        if i + 2 < len(gsl_d[3]): #dont go out of bounds
            temp_max = find_last_element(i, gsl_d)
            temp_min = find_first_element(i, gsl_d)
            #if gsl_d[3,i] == 14:
                #print(temp_min, ' ', temp_max, ' ', i)
#             if gsl_d[3,i] == 41:
#                 print(temp_min, ' ', temp_max, ' ', i)
            gsl_d[3,temp_min:temp_max] = gsl_d[3,i] 
#             if gsl_d[3,i] == 41:
#                 print(temp_min, ' ', temp_max, ' ', i, 'AFTER ASSIGNMENT')
    return gsl_d
#tex = remove_overlap(gsla)

In [16]:
#label 14 goes till 14564, 41 starts at 14482
    #so there is an overlap 

In [17]:
def create_confusion_array(gsl_d, gsla):
    confusion_value = []
    confusion_label = []
    for i in range(len(gsla[3])):
        if gsl_d[3,i] != gsla[3,i]:
            #print('im confused')
            temp = [gsl_d[3,i]]
            confusion_value.append(temp)
            confusion_label.append(i)
    confusion_value = np.asarray(confusion_value)
    confusion_label = np.asarray(confusion_label)
    confusion_array = np.column_stack((confusion_value, confusion_label))

    return confusion_array

In [18]:
#gets the last index with specific label, (this function is called iteravely)
def get_last_time(label, gsl_arr, displace):
    #print('label: ', label)
    for i in reversed(range(len(gsl_arr[3]))): 
        #print(i)
        if int(gsl_arr[3,i]) == int(label):
#             if label == 41:
#                 print('i: ', i, ' label: ', label, ' gls_arr: ', gsl_arr[3,i] )
#                 max_index = i
#                 print('\n, type of return val: ', type(max_index - displace), max_index - displace)
#                 return max_index - displace
            max_index = i
            return max_index - displace

In [19]:
# #Finally found the source of the issue (remove_overlap)
# np.where( gsla_no_overlap[3] == 41 )
# np.unique(gsla_no_overlap[3])
# np.unique(gsla[3])

In [20]:
#gets the first index with specific label, (this function is called iteravely)
def get_first_time(label, gsl_arr, displace): 
    for i in range( len(gsl_arr[3])):
        if gsl_arr[3,i]  == 41:#int(label):
            min_index = i
#             print(gsl_arr[3,i])
#             print('\n type of return val: ', type(min_index - displace), min_index - displace, ' i: ', i, ' label: ', label)
            return min_index + displace

In [21]:
# len(gsla_no_overlap[3])

In [22]:
#gets the total duration for each label,
#this is the portion of the code I am still working on, 
#something with indexes has gone wrong in the 'get_last_time'
#it works except for when the label is 0 
def create_absolute_time(gsl_arr, disp):
    disp = int(disp)
    gsla_no_overlap = np.copy(gsl_arr)
    abs_time_value = []
    abs_time_label = []
    abs_time_time  = []
    for i in range(50):
        abs_time_value.append(get_last_time(i, gsla_no_overlap, disp))
#         temp_gsla = np.copy(gsla_no_overlap)
#         abs_time_value.append(get_first_time(i, np.flip(temp_gsla,1), disp))
        abs_time_label.append(get_first_time(i, gsla_no_overlap, disp))  
        abs_time_time.append(i)
        
    abs_time_value = np.asarray(abs_time_value)
    abs_time_label = np.asarray(abs_time_label)
    abs_time_time = np.asarray(abs_time_time)
    abs_time_array = np.column_stack((abs_time_value, abs_time_label, abs_time_time))
    combined_time = np.zeros(shape=(len(abs_time_array[:,0]),4), dtype=np.int32, order='C')
    combined_time[:,:] = -1
    #print(abs_time_array[:51,:3])
    #combined_time[:,:3] = abs_time_array[:,:3]
    try:
        combined_time[:,:3] = abs_time_array[:,:3]
        
    except:#hardcoding temporary solutions on the current dataset
           #I know this is heresy, I promise to fix it soon
        print('None type found...')
        if  abs_time_array[40,0] == None: 
            abs_time_array[40,0] = 31445
            abs_time_array[40,1] = 31186
        if  abs_time_array[25,0] == None:# 28415  28539
            abs_time_array[25,0] = 28539
            abs_time_array[25,1] = 28415
        if  abs_time_array[21,0] == None:# 29685   30008
            abs_time_array[21,0] = 30008
            abs_time_array[21,1] = 29685
        if  abs_time_array[41,0] == None:#   31837  31898
            abs_time_array[41,0] = 31898
            abs_time_array[41,1] = 31837
        combined_time[:,:3] = abs_time_array[:,:3]
    for i in range(len(abs_time_array)):
        combined_time[i,3] = abs_time_array[i,0] - abs_time_array[i,1]
    return combined_time
create_absolute_time(gsla_no_overlap, 0)

NameError: name 'gsla_no_overlap' is not defined

In [ ]:
#add each focus duration succesively 
def create_sequential_sum(arr):
    arr = np.copy(arr)
    my_sum = 0
    for i in range(len(arr)):
        my_sum = my_sum + arr[i,3]
        arr[i,3] = my_sum
    return arr

In [ ]:
#this is essentially the main, "Class", in that from here all other
#functions are called and variables we need are kept here.  
sid = ["3"]#,"77"] #["25","78"]
#done subjects: 43, 78, 25,
stim_idx = ["1","2","3","4"] 

tmp_path = 'D:/Documents/My_Code/Christoforos/temp_data_for_Walker'
directory = 'D:/Documents/My_Code/Christoforos/temp_data_for_Walker'

for sid_ in sid:
    for stim_ in stim_idx:
        filename = "D:/Documents/My_Code/Christoforos/temp_data_for_Walker/gaze_sid_" + sid_ + "_stimid_" + stim_ + ".pickle"
        with open(filename, 'rb') as f:
            timestamp, gaze, fixations  =  pickle.load(f)

        gaze_stamp = initialize_gaze_stamp()
        initial_vis()
        dbscan_labels = my_dbscan()
        gsln = create_gsln(dbscan_labels)
        agg_labels = agglom_clustering(gsln)
        gsla = gsln 
        gsla[3] = agg_labels

        gsla_no_overlap = np.copy(remove_overlap(gsla))
        
        reset_save("abs_time_0_displace_"+sid_+stim_)
        disp = 0
        abs_time = create_absolute_time(gsla_no_overlap, disp)
        save_results(abs_time, "abs_time_0_displace_"+sid_+stim_)

#This section can be run completeley on its own (No need to re-cluster etc.)

#This section gets the absolute fixation times according to stimulus type
    #also does some statistics and plotting. 
abs_time_list_difficult_rhyme = []
abs_time_list_difficult_visual = []
abs_time_list_easy_rhyme = []
abs_time_list_easy_visual = []
disp = "0" # or "3"
sid = ["43","25"]# ["3","25","27","29","43","77","78"]
stim_idx = ["1","2","3","4"]
for sid_ in sid:
    for stim_ in stim_idx:
        if stim_ == "1":
            abs_time_list_difficult_rhyme = []
            abs_time_list_difficult_rhyme.append(load_results("abs_time_"+disp+"_displace_"+sid_+stim_))
            abs_time_arr_difficult_rhyme = np.asarray(abs_time_list_difficult_rhyme[0])   
            abs_seq_difficult_rhyme  = create_sequential_sum(abs_time_arr_difficult_rhyme)#[0]
            plt.scatter(abs_seq_difficult_rhyme[:,2], abs_seq_difficult_rhyme[:,3], marker='^')          
        if stim_ == "2":             
            abs_time_list_easy_rhyme = []
            abs_time_list_easy_rhyme.append(load_results("abs_time_"+disp+"_displace_"+sid_+stim_))
            abs_time_arr_easy_rhyme = np.asarray(abs_time_list_easy_rhyme[0])   
            abs_seq_easy_rhyme  = create_sequential_sum(abs_time_arr_easy_rhyme)#[0]
            plt.scatter(abs_seq_easy_rhyme[:,2], abs_seq_easy_rhyme[:,3], marker='o')
            plt.title('subject id: '+sid_+ ' rhyme confound')
            plt.legend(('Difficult Test', 'Easy Test', 'Masked if < -0.5'),
                   loc='upper left')
            #this seperates the plots on the 'stim' attribute 
            plt.figure()
            
        if stim_ == "3":
            abs_time_list_difficult_visual = []
            abs_time_list_difficult_visual.append(load_results("abs_time_"+disp+"_displace_"+sid_+stim_))
            abs_time_arr_difficult_visual = np.asarray(abs_time_list_difficult_visual[0])   
            abs_seq_difficult_visual  = create_sequential_sum(abs_time_arr_difficult_visual)#[0]
            plt.scatter(abs_seq_difficult_visual[:,2], abs_seq_difficult_visual[:,3], marker='^')
        if stim_ == "4":
            abs_time_list_easy_visual = []
            abs_time_list_easy_visual.append(load_results("abs_time_"+disp+"_displace_"+sid_+stim_))
            abs_time_arr_easy_visual = np.asarray(abs_time_list_easy_visual[0])   
            abs_seq_easy_visual  = create_sequential_sum(abs_time_arr_easy_visual)#[0]
            plt.scatter(abs_seq_easy_visual[:,2], abs_seq_easy_visual[:,3], marker='o')
            plt.title('subject id: '+sid_+ ' visual confound')
            plt.legend(('Difficult Test', 'Easy Test', 'Masked if < -0.5'),
                   loc='upper left')
            plt.figure()
print(sid_,' rhyme: ', stats.ttest_ind(abs_seq_difficult_rhyme,abs_seq_easy_rhyme))
print(sid_,' rhyme: ', stats.ttest_ind(abs_seq_difficult_visual,abs_seq_easy_visual),'\n')